In [3]:
# 라이브러리 임포트
import os
import pandas as pd
import folium
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px  # Import plotly express






# Folium 지도 위경도 데이터
file = pd.read_csv('D:/yeo_pm.csv', encoding='cp949')
station = pd.DataFrame(file)

# Folium 지도
initial_location = [37.525989, 126.923948]
my_map = folium.Map(location=initial_location, zoom_start=14)
for index, row in station.iterrows():
    lat, lon = row['X'], row['Y']
    folium.Marker(location=[lat, lon], popup=row['대여소명']).add_to(my_map)


# 파일이 있는 기본 경로
base_path = 'D:/'

rental_data = {
    f'{month}월 시간대별 대여현황':
        {
            os.path.join(base_path, f'cleanup-main/tpss_bcycl_od_statnhm_2023{month:02d}{day:02d}.csv').replace("\\", ""): f'{month:02d}/{day:02d}' 
            for day in range(1, 32) if day <= (30 if month in {4, 6} else 28) # if month == 2 else 31)
        }
    for month in range(1, 7)
}






# 파일 데이터 불러오기(따릉이 자치구별 대여량)
file_paths = {
    '1월 대여소별 대여현황': 'D:/PM_CNT_1.csv',
    '2월 대여소별 대여현황': 'D:/PM_CNT_2.csv',
    '3월 대여소별 대여현황': 'D:/PM_CNT_3.csv',
    '4월 대여소별 대여현황': 'D:/PM_CNT_4.csv',
    '5월 대여소별 대여현황': 'D:/PM_CNT_5.csv',
    '6월 대여소별 대여현황': 'D:/PM_CNT_6.csv'
}




# 대시 및 레이아웃 구성
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("여의도 내 따릉이 대여소 및 대여현황"),
    #  여의도내 따릉이 대여소 위치현황
    html.Div([
        html.Iframe(id='map', srcDoc=my_map._repr_html_(), width='100%', height='600'),
        html.P("주제: 여의도 내 지하철역과 서울시 공공자전거 정류소의 이용 현황 분석을 통한 교통 분담률 개선 방안"),
        html.P("출퇴근 시간대에 유난히 몰리는 교통 수단의 분담률을 개선하면 어떨까?라는 의문에서 조사를 시작했다."),
        html.P("조사대상지: 서울특별시 영등포구 여의도 일대"),
        html.P("조사 대상지인 여의도 일대와 따릉이 대여소 현황이다."),
        html.P("현재 43개의 정류소가 여의도 내 위치하고 있다."),
        html.P("다음은 대상지 선정과정이다.")
    ]),
    
    

  html.Div([
    html.H2("따릉이 시간대별 대여현황"),
    dcc.Dropdown(
        id='file-dropdown',
        options=[
            {'label': key, 'value': key} for key in rental_data.keys()
        ],
        value=list(rental_data.keys())[1],  # 초기값을 두 번째 키로 설정
        style={'width': '50%'}
    ),
    dcc.RadioItems(
        id='weekday-weekend-radio',
        options=[
            {'label': '평일', 'value': '평일'},
            {'label': '주말', 'value': '주말'}
        ],
        value='평일',
        style={'margin-top': '10px'}
    ),
    dcc.Graph(id='line-plot'),
    html.P("서울열린데이터 광장에서 수집한 2023년 1월1일부터 6월30일까지의 시간대별 따릉이 이용 데이터다."
           "출퇴근 시간대에 높은 첨두교통량을 보인다. 따라서 출퇴근 용도로 따릉이를 많이 사용한다는 가설이 도출된다."
           "주말과 평일을 비교한다면 평일에 이용량이 더 많은 모습을 보인다."
           "추가로 출근시간대에 급격히 이용량이 증가하고 감소하는 모습이 보인다."
           "그러나 퇴근시간대에 급격한 이용량 증가 후 감소폭은 출근시간대에 비해 완만하다."
           "비가 오거나 미세먼지 농도가 높은 날은(ex: 04/05, 05/06) 이용량이 대폭 감소한다."
           "1월에서 6월까지 날씨가 온난해짐에 따라 전체적인 이용량이 증가한다."
           "위 사례를 통해 자전거 이동수단의 특성과 이용용도를 파악할 수 있다."
           "이와 같은 조사를 통해 조사 대상지를 출퇴근이 많이 발생하는 곳으로 후보군을 선정했다."
           "서울시내 중심업무지구 '종로, 강남, 여의도'를 선정했다."),
]),


   
    
    
    
    
    
   
    
    
    
    
    
    
    
    
    # 따릉이 자치구별 대여현황
    html.Div([
        html.H2("따릉이 자치구별 대여현황"),
        dcc.Dropdown(
            id='month-dropdown',
            options=[{'label': month, 'value': month} for month in file_paths.keys()],
            value='1월 대여소별 대여현황',
            style={'width': '50%'}
        ),
        dcc.Graph(id='bar-chart'),
        html.P("서울열린데이터 광장에서 수집한 2023년 1월부터 6월까지 자치구별 따릉이 대여현황이다."
               "위 데이터에 따르면 강서구, 송파구, 영등포구 순으로 이용량이 많음을 알 수 있다."
               "따라서 두 조건을 모두 만족하는 영등포구 여의도 일대를 조사 대상지로 선정했다."
               "여의도의 경우 따릉이를 가장 먼저 도입한 시범지역이며 한강으로 둘러싸여있어 조사대상지를 명확히 구분할 수 있다."),
    ]),








 # HTML 파일 추가
    html.Div([
    ]),      

     html.Div([
        html.Iframe(src='https://seung-yong-lee.github.io/ubps/자전거.html', width='100%', height='600')
    ]),    
     html.Div([
        html.Iframe(src='https://seung-yong-lee.github.io/ubps/지하철 수정.html', width='100%', height='600')
    ]),
     
    
        html.Div([
        html.Iframe(src='https://seung-yong-lee.github.io/ubps/distance_map_with_labelsL.html', width='100%', height='600')
    ]),
 
        html.Div([
        html.Iframe(src='https://seung-yong-lee.github.io/ubps/따릉이 정류소간 인접.html', width='100%', height='600')
    ]),
       
        html.Div([
        html.Iframe(src='https://seung-yong-lee.github.io/ubps/map2.html', width='100%', height='600')
    ]),
       
            html.Div([
        html.Iframe(src='https://seung-yong-lee.github.io/ubps/map_with_red_blue_green_buffers1.html', width='100%', height='600')
    ]),
               
     

  
         html.Div([
    html.Div([
        html.P("도시 교통 효율성 강화를 위한 환승체계 도입에 대한 현황 분석과 결론",
               style={'font-size': '24px', 'font-weight': 'bold', 'text-align': 'center', 'color': 'darkblue'})
    ]),

    html.Div([
        html.H4("현황분석", style={'color': 'darkblue'}),
        html.P("1. 따릉이의 대여 및 반납 건수\n따릉이의 높은 대여 및 반납 건수는 시민들이 이 교통 수단을 적극적으로 활용하고 있다는 긍정적인 신호이다. 대여 및 반납 건수의 늘어남은 도시 내 이동 수요가 높아지고 있다는 것을 시사한다."),
        html.P("2. 지하철역의 승차승객수와 하차승객수\n지하철역의 승하차승객수가 높은 것은 대중교통이 활발히 이용되고 있음을 보여준다. 이는 지하철을 중심으로 한 교통 흐름이 상당히 활성화되어 있다는 의미이다."),
        html.P("3. 따릉이 군집의 현황\n지하철역 주변에 도보권으로 접근 가능한 따릉이 군집이 형성되어 있어, 지하철과 따릉이 간의 자연스러운 연계성이 있음을 확인할 수 있었다.")
    ]),

    html.Div([
        html.H4("결론", style={'color': 'darkblue'}),
        html.P("1. 따릉이와 지하철 간의 연계성이 높아짐으로써 출퇴근 시간대에 교통 수단 분담률이 개선될 것으로 기대된다. 이를 통해 교통 분산 효과를 얻어, 교통 정체 현상을 방지하고 교통 체계의 부담을 분산시키는 데 도움이 될 것이다."),
        html.P("2. 따릉이와 지하철의 높은 이용률은 도시 교통 수요가 높아져 있음을 시사한다."),
        html.P("3. 따릉이의 환승체계 도입은 교통 수단 분담률 개선 방안 중 하나가 된다. 또한 이러한 도시 교통의 현황에 맞는 또다른 대안이 될 수 있다."),
        html.P("4. 지하철과 따릉이의 연계성을 높여 교통 이용자들에게 더욱 편리하고 효율적인 이동을 제공함으로써, 도시 교통의 효율성을 향상시키는 데 중요한 역할을 할 것으로 기대된다.")
    ]),
             
    html.Div([
        html.H4("출처", style={'color': 'darkblue'}),
        html.P("1. -따릉이 대여소 지도: 서울열린데이터 광장, 서울시 따릉이 대여소 마스터 정보(https://data.seoul.go.kr/dataList/OA-21235/S/1/datasetView.do)"),
        html.P("   -따릉이 시간대별 그래프: 서울열린데이터 광장, 서울시 따릉이 대여소별 대여/반납 승객수(https://data.seoul.go.kr/dataList/OA-21229/F/1/datasetView.do)"),
        html.P("   -따릉이 자치구별 대여현황 그래프: 서울열린데이터광장, 서울시 공공자전거 실시간 대여정보(https://data.seoul.go.kr/dataList/OA-15493/A/1/datasetView.do)"),
        html.P("2. -공공자전거와 지하철의 이용합계 위치도(지하철 승차 및 하차 승객수): 티머니 대중교통 통계자료, 2023년 01~06월 교통카드 통계자료(https://www.t-money.co.kr/ncs/pct/ugd/ReadTrcrStstList.dev)"),
        html.P("   -공공자전거와 지하철의 이용합계 위치도(지하철 위경도값):, 서울열린데이터광장, 서울시 역사마스터 정보(https://data.seoul.go.kr/dataList/OA-21232/S/1/datasetView.do)"),
        html.P("   -공공자전거와 지하철의 이용합계 위치도(따릉이 대여 및 반납 건수):, 서울열린데이터광장, 서울시 공공자전거 대여소별 이용정보(월별)(https://data.seoul.go.kr/dataList/OA-15249/F/1/datasetView.do)"),
        html.P("   -공공자전거와 지하철의 이용합계 위치도(따릉이 위경도값):, 서울열린데이터광장, 서울시 공공자전거 실시간 대여정보(https://data.seoul.go.kr/dataList/OA-15493/A/1/datasetView.do)"),
        html.P(""),
        html.P("    기타 자료는 위의 데이터들을 활용")
    ])         
])

])    
    
    


#콜백구성
# 시간대별 대여현황 콜백
@app.callback(
    Output('line-plot', 'figure'),
    [Input('file-dropdown', 'value'),
     Input('weekday-weekend-radio', 'value')]
)
def update_line_plot(selected_mapping, weekday_weekend):
    combined_data = pd.DataFrame()

    for filename, legend_label in rental_data[selected_mapping].items():
        try:
            data = pd.read_csv(filename, encoding='cp949')
            # '기준_날짜' 열이 이미 날짜 형식으로 되어 있을 경우, 추가 변환 수환 x
            if '기준_날짜' not in data.columns:
                raise ValueError("")
        except Exception as e:
            print(f"{e}")
            continue

        # '기준_날짜' 열이 날짜 데이터 형식이 아닌 경우 변환 수행
        if not pd.api.types.is_datetime64_any_dtype(data['기준_날짜']):
            data['기준_날짜'] = pd.to_datetime(data['기준_날짜'], format='%Y%m%d')

        # 선택한 옵션에 따라 주중 또는 주말을 필터링
        if weekday_weekend == '평일':
            data = data[data['기준_날짜'].dt.weekday < 5]  # 월요일부터 금요일까지가 주중 (0부터 4)
        elif weekday_weekend == '주말':
            data = data[data['기준_날짜'].dt.weekday >= 5]  # 토요일과 일요일이 주말 (5와 6)

        rental_count_by_hour = data.groupby('기준_시간대').size()
        combined_data[legend_label] = rental_count_by_hour

    # Calculate the overall average
    combined_data['평균 대여량'] = combined_data.mean(axis=1)

    fig = px.line(combined_data, x=combined_data.index, y=combined_data.columns, markers=True)

    # '평균 대여량'의 선과 점 굵기와 색상 조절
    fig.update_traces(selector=dict(name='평균 대여량'), line=dict(width=5, color='black'), marker=dict(size=7))

    fig.update_layout(
        title=f'{weekday_weekend} 시간대별 따릉이 대여현황',
        xaxis_title='시간대',
        yaxis_title='대여 횟수',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        )

    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

    return fig



# 자치구별 대여현황 콜백
@app.callback(Output('bar-chart', 'figure'),[Input('month-dropdown', 'value')])
    
    

def update_chart(selected_month):
    file_path = file_paths[selected_month]  # 해당 월에 대한 파일 경로 가져오기
    data = pd.read_csv(file_path, encoding='cp949')

    # Group by '자치구' and sum '이용합계'
    grouped_data = data.groupby('자치구')['이용합계'].sum().reset_index()

    fig = px.bar(grouped_data, x='자치구', y='이용합계', title=f'따릉이 {selected_month}', labels={'이용합계': '이용 합계'})
    return fig

# 서버를 실행하기 전에 에러 핸들러 설정
app.server.errorhandler(Exception)
def _wrap_errors(error):
    print(f"에러 발생: {error}")
    return "에러가 발생했습니다. 자세한 내용은 로그를 확인하세요.", 500

if __name__ == '__main__':
    app.run_server(debug=True, port=13293)

